In [1]:
import os
import sys

# Save current working directory
cwd = os.getcwd()

# Change to parent directory
parent_dir = os.path.abspath(os.path.join(cwd, '..'))
os.chdir(parent_dir)

# Temporarily add parent directory to sys.path
sys.path.insert(0, parent_dir)
import framework
sys.path.pop(0)

# Return to original directory
os.chdir(cwd)

In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import vit_b_32, ViT_B_32_Weights
from torch import nn, optim
from tqdm import tqdm

weights = ViT_B_32_Weights.DEFAULT
# Transform matching ImageNet-trained model
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    weights.transforms()
])

model = vit_b_32()
model.heads = nn.Linear(in_features = 768, out_features = 100, bias = True)

device = "cuda"
state_dict = torch.load('model_weights.pth', map_location=device)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [3]:
test_data = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_data, batch_size=64)

from framework.criterion import classification_accuracy_loader

classification_accuracy_loader(model, test_loader, device)

Files already downloaded and verified


c:\Repositories\seu-injection-framework\.venv\Lib\site-packages\torch\nn\modules\activation.py:1196: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  return torch._native_multi_head_attention(


0.1575

In [4]:
from framework.attack import Injector

inj = Injector(model, classification_accuracy_loader, device=device, data_loader=test_loader)

Testing a forward pass on cuda...


In [5]:
# layer_name = "heads.bias"

results = inj.run_stochastic_seu(1,1/10_000_000)

results

Testing Layer:  class_token


768it [00:00, ?it/s]


Testing Layer:  conv_proj.weight


2359296it [00:03, 657015.10it/s]


Testing Layer:  conv_proj.bias


...

Testing Layer:  heads.weight


76800it [00:00, 481117.42it/s]


Testing Layer:  heads.bias


100it [00:00, 103563.06it/s]


{'tensor_location': [(851, 550),
  (305, 203),
  (755, 1899),
  (174, 694),
  (1088, 340),
  (467, 724),
  (543, 598),
  (795, 420),
  (1453, 185)],
 'criterion_score': [0.1554,
  0.01,
  0.01,
  0.1576,
  0.1575,
  0.01,
  0.01,
  0.0772,
  0.0435],
 'layer_name': ['encoder.layers.encoder_layer_2.self_attention.in_proj_weight',
  'encoder.layers.encoder_layer_2.self_attention.out_proj.weight',
  'encoder.layers.encoder_layer_2.mlp.3.weight',
  'encoder.layers.encoder_layer_4.self_attention.in_proj_weight',
  'encoder.layers.encoder_layer_8.self_attention.in_proj_weight',
  'encoder.layers.encoder_layer_8.mlp.3.weight',
  'encoder.layers.encoder_layer_8.mlp.3.weight',
  'encoder.layers.encoder_layer_9.mlp.0.weight',
  'encoder.layers.encoder_layer_11.mlp.0.weight'],
 'value_before': [-0.0009086228,
  -0.0034944166,
  -0.010906478,
  0.15107386,
  -0.012796498,
  0.11468512,
  -0.00087544793,
  -0.008881114,
  0.06358506],
 'value_after': [-3.0918832015976978e+35,
  -1.1890883504198975e

In [6]:
import pandas as pd

pd.DataFrame(results)

,tensor_location,criterion_score,layer_name,value_before,value_after
0,"(851, 550)",0.1554,encoder.layers.encoder_layer_2.self_attention....,-0.000909,-3.091883e+35
1,"(305, 203)",0.0100,encoder.layers.encoder_layer_2.self_attention....,-0.003494,-1.189088e+36
2,"(755, 1899)",0.0100,encoder.layers.encoder_layer_2.mlp.3.weight,-0.010906,-3.711282e+36
3,"(174, 694)",0.1576,encoder.layers.encoder_layer_4.self_attention....,0.151074,5.140777e+37
4,"(1088, 340)",0.1575,encoder.layers.encoder_layer_8.self_attention....,-0.012796,-4.354423e+36
5,"(467, 724)",0.0100,encoder.layers.encoder_layer_8.mlp.3.weight,0.114685,3.902532e+37
6,"(543, 598)",0.0100,encoder.layers.encoder_layer_8.mlp.3.weight,-0.000875,-2.978995e+35
7,"(795, 420)",0.0772,encoder.layers.encoder_layer_9.mlp.0.weight,-0.008881,-3.022087e+36
8,"(1453, 185)",0.0435,encoder.layers.encoder_layer_11.mlp.0.weight,0.063585,2.163687e+37
